In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch

name = './../model/LabVGG16.pth'
dictname = name[:-4] + '_dict' + name[-4:]
print(dictname)

net = torch.load(name,map_location=torch.device('cpu'))
dictnet = net.state_dict()
torch.save(dictnet, dictname)

./../model/LabVGG16_dict.pth


LAB VGG16: Trying to import up to pool2

In [2]:
import torch
import torch.nn as nn

def l2normalize(v, eps = 1e-12):
    return v / (v.norm() + eps)


class LabVGG16(nn.Module):
    def __init__(self, in_dim = 3, num_classes = 1000):
        super(LabVGG16, self).__init__()
        # feature extraction part
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = in_dim, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        )
        self.pool1 = nn.Sequential(
            nn.ReLU(inplace = False),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)
        )
        self.pool2 = nn.Sequential(
            nn.ReLU(inplace = False),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        """ self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1)
        )
        self.pool3 = nn.Sequential(
            nn.ReLU(inplace = False),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1)
        )
        self.pool4 = nn.Sequential(
            nn.ReLU(inplace = False),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1)
        )
        self.pool5 = nn.Sequential(
            nn.ReLU(inplace = False),
            nn.AdaptiveAvgPool2d((7, 7))
        )
        # classification part
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        ) """

    def forward(self, x):                                   # shape: [B, 3, 224, 224]
        conv1 = self.conv1(x)                               # shape: [B, 64, 224, 224]
        pool1 = self.pool1(conv1)                           # shape: [B, 64, 112, 112]
        conv2 = self.conv2(pool1)                           # shape: [B, 128, 112, 112]
        pool2 = self.pool2(conv2)                           # shape: [B, 128, 56, 56]
        """ conv3 = self.conv3(pool2)                           # shape: [B, 256, 56, 56]
        pool3 = self.pool3(conv3)                           # shape: [B, 256, 28, 28]
        conv4 = self.conv4(pool3)                           # shape: [B, 512, 28, 28]
        pool4 = self.pool4(conv4)                           # shape: [B, 512, 14, 14]
        conv5 = self.conv5(pool4)                           # shape: [B, 512, 14, 14]
        pool5 = self.pool5(conv5)                           # shape: [B, 512, 7, 7]
        pool5 = pool5.view(x.size(0), -1)                   # shape: [B, 512 * 7 * 7]
        x = self.classifier(pool5)                          # shape: [B, 1000]
        return x """
        return pool2

net= LabVGG16()

In [3]:
# Load the state dictionary
state_dict = torch.load("./../model/LabVGG16_dict.pth", map_location=torch.device('cpu'))

In [4]:
count=0
for key in state_dict.keys():
    print("{}. {}".format(count,key))
    count+=1

0. conv1.0.weight
1. conv1.0.bias
2. conv1.1.weight
3. conv1.1.bias
4. conv1.1.running_mean
5. conv1.1.running_var
6. conv1.1.num_batches_tracked
7. conv1.3.weight
8. conv1.3.bias
9. pool1.0.weight
10. pool1.0.bias
11. pool1.0.running_mean
12. pool1.0.running_var
13. pool1.0.num_batches_tracked
14. conv2.0.weight
15. conv2.0.bias
16. conv2.1.weight
17. conv2.1.bias
18. conv2.1.running_mean
19. conv2.1.running_var
20. conv2.1.num_batches_tracked
21. conv2.3.weight
22. conv2.3.bias
23. pool2.0.weight
24. pool2.0.bias
25. pool2.0.running_mean
26. pool2.0.running_var
27. pool2.0.num_batches_tracked
28. conv3.0.weight
29. conv3.0.bias
30. conv3.1.weight
31. conv3.1.bias
32. conv3.1.running_mean
33. conv3.1.running_var
34. conv3.1.num_batches_tracked
35. conv3.3.weight
36. conv3.3.bias
37. conv3.4.weight
38. conv3.4.bias
39. conv3.4.running_mean
40. conv3.4.running_var
41. conv3.4.num_batches_tracked
42. conv3.6.weight
43. conv3.6.bias
44. pool3.0.weight
45. pool3.0.bias
46. pool3.0.running_

In [5]:
first_28_keys = list(state_dict.keys())[:28]
extracted_dict = {key: state_dict[key] for key in first_28_keys}
count=0
for key in extracted_dict.keys():
    print("{}. {}".format(count,key))
    count+=1

0. conv1.0.weight
1. conv1.0.bias
2. conv1.1.weight
3. conv1.1.bias
4. conv1.1.running_mean
5. conv1.1.running_var
6. conv1.1.num_batches_tracked
7. conv1.3.weight
8. conv1.3.bias
9. pool1.0.weight
10. pool1.0.bias
11. pool1.0.running_mean
12. pool1.0.running_var
13. pool1.0.num_batches_tracked
14. conv2.0.weight
15. conv2.0.bias
16. conv2.1.weight
17. conv2.1.bias
18. conv2.1.running_mean
19. conv2.1.running_var
20. conv2.1.num_batches_tracked
21. conv2.3.weight
22. conv2.3.bias
23. pool2.0.weight
24. pool2.0.bias
25. pool2.0.running_mean
26. pool2.0.running_var
27. pool2.0.num_batches_tracked


In [6]:
net.load_state_dict(extracted_dict, strict=False)

_IncompatibleKeys(missing_keys=['conv1.2.weight', 'conv1.2.bias', 'conv2.2.weight', 'conv2.2.bias'], unexpected_keys=['conv1.3.weight', 'conv1.3.bias', 'conv1.1.weight', 'conv1.1.bias', 'conv1.1.running_mean', 'conv1.1.running_var', 'conv1.1.num_batches_tracked', 'pool1.0.weight', 'pool1.0.bias', 'pool1.0.running_mean', 'pool1.0.running_var', 'pool1.0.num_batches_tracked', 'conv2.3.weight', 'conv2.3.bias', 'conv2.1.weight', 'conv2.1.bias', 'conv2.1.running_mean', 'conv2.1.running_var', 'conv2.1.num_batches_tracked', 'pool2.0.weight', 'pool2.0.bias', 'pool2.0.running_mean', 'pool2.0.running_var', 'pool2.0.num_batches_tracked'])

In [7]:
print(net.conv1[2].weight)

Parameter containing:
tensor([[[[-1.6060e-02,  3.5647e-02, -2.6465e-02],
          [-3.4588e-02,  4.1321e-02,  2.2004e-02],
          [-2.3385e-02, -1.3899e-02,  6.0169e-03]],

         [[-4.0315e-02,  2.0721e-02, -2.9332e-02],
          [-2.4635e-03, -3.9853e-02, -4.1471e-02],
          [ 2.0366e-02, -2.7438e-02, -3.7518e-02]],

         [[-3.2807e-02,  3.9115e-02, -2.1105e-02],
          [-3.1494e-02, -1.9086e-03,  2.2245e-02],
          [-1.4164e-02, -2.4516e-02, -3.2719e-02]],

         ...,

         [[ 5.4701e-03, -1.8519e-02,  3.5023e-02],
          [ 1.0928e-02, -1.3364e-02, -1.8104e-02],
          [ 2.0138e-02, -2.1494e-02,  4.1186e-02]],

         [[ 3.8577e-02, -4.1858e-03, -1.1301e-02],
          [-2.3392e-02, -1.3369e-03,  5.6111e-03],
          [ 2.0366e-02, -3.8379e-02,  2.6027e-02]],

         [[ 4.1221e-02,  3.9964e-02, -3.3387e-02],
          [ 3.1966e-02, -7.9190e-03,  3.5187e-02],
          [-4.0758e-02,  2.9253e-02, -1.9972e-02]]],


        [[[ 2.7099e-02, -8.1451

Already working: Load entire dictionary

In [8]:
# To load entire state dictionary
net.load_state_dict(torch.load("./../model/LabVGG16_dict.pth",map_location=torch.device('cpu')))

RuntimeError: Error(s) in loading state_dict for LabVGG16:
	Missing key(s) in state_dict: "conv1.2.weight", "conv1.2.bias", "conv2.2.weight", "conv2.2.bias". 
	Unexpected key(s) in state_dict: "conv3.0.weight", "conv3.0.bias", "conv3.1.weight", "conv3.1.bias", "conv3.1.running_mean", "conv3.1.running_var", "conv3.1.num_batches_tracked", "conv3.3.weight", "conv3.3.bias", "conv3.4.weight", "conv3.4.bias", "conv3.4.running_mean", "conv3.4.running_var", "conv3.4.num_batches_tracked", "conv3.6.weight", "conv3.6.bias", "pool3.0.weight", "pool3.0.bias", "pool3.0.running_mean", "pool3.0.running_var", "pool3.0.num_batches_tracked", "conv4.0.weight", "conv4.0.bias", "conv4.1.weight", "conv4.1.bias", "conv4.1.running_mean", "conv4.1.running_var", "conv4.1.num_batches_tracked", "conv4.3.weight", "conv4.3.bias", "conv4.4.weight", "conv4.4.bias", "conv4.4.running_mean", "conv4.4.running_var", "conv4.4.num_batches_tracked", "conv4.6.weight", "conv4.6.bias", "pool4.0.weight", "pool4.0.bias", "pool4.0.